In [ ]:
import pandas as pd
import numpy as np
#import time
#import scipy.sparse as sparse
#from scipy.sparse.linalg import spsolve
from sklearn import metrics
#import random
import os

In [ ]:
import implicit

In [1]:
class Implicit:
    def __init__(self, seed, environment = "offline"):
        self.seed = seed
        
        if environment == "kaggle":
            self.read_matrices_kaggle()
        else:
            self.read_matrices()
        
    def read_matrices(self):        
        self.matrix_asFrame = pd.read_excel(f"../data.xlsx")
        self.matrix = sparse.load_npz(f"../ratings.npz")
        
        self.num_users, self.num_items = self.matrix.shape
    
    def read_matrices_kaggle(self):
        self.matrix_asFrame = pd.read_excel(f"/kaggle/input/data.xlsx")
        self.matrix = sparse.load_npz(f"/kaggle/input/ratings.npz")
        
        self.num_users, self.num_items = self.matrix.shape
        
    def create_training_data(self, percent_test_set = 0.15):
        training_set = self.matrix.copy()
        positive_indices = training_set.nonzero()
        positive_values = list(zip(positive_indices[0], positive_indices[1]))
        test_set = self.matrix.copy()
        # The test set should only measure, if the item is going to be bought, not the frequency.
        test_set[test_set != 0] = 1
        random.seed(self.seed)
        n_samples = int(percent_test_set*len(positive_values))
        samples = random.sample(positive_values, n_samples)
        # Put all chosen samples into lists.
        user_indices = [index[0] for index in samples]
        item_indices = [index[1] for index in samples]
        # All users and items that are used in the test set, should be unviewed in the training set.
        training_set[user_indices, item_indices] = 0
        training_set.eliminate_zeros()
        
        self.training_set = training_set
        self.test_set = test_set
        self.indices = list(set(user_indices))
    
    def alternating_least_squares(self, iterations, factors, i_lambda, alpha):
        self.user_vecs, self.item_vecs = implicit.alternating_least_squares((self.training_set*alpha).astype('double'), factors=factors, regularization = i_lambda, iterations = iterations)
    
    def predict_user_byIndex(self, index):
        return user_vecs[index,:].dot(item_vecs.transpose())

In [ ]:
Factorizer = Matrix_Factorizer(941, "offline")
Factorizer.create_training_data()

In [ ]:
# In Kaggle Kernels
Factorizer = Matrix_Factorizer(941, "kaggle")
Factorizer.create_training_data()

In [ ]:
Factorizer.alternating_least_squares(iterations = 10, factors = 40, i_lambda = 0.15, alpha = 30)

In [ ]:
Factorizer.predict_user_byIndex(10)